In [187]:
import numpy as np
import numpy.linalg as la

# Gaussian elimination (assuming pivot elements are nonzero!)
def GE(B):
    A = np.copy(B) # make a copy to avoid changing matrix passed to the function
    m,n = np.shape(B) # allow for augmented matrix [A B], where A is mxm, while B is (n-m)x(n-m) and may be empty 0x0
    for k in range(0,m-1):
        for j in range(k+1,m):
            l = A[j,k]/A[k,k]
            for p in range(k,n):
                A[j,p] -= l*A[k,p]
    return A


# Gaussian elimination with partial pivoting
def GEPP(B):
    A = np.copy(B)
    m,n = np.shape(A) # allow for augmented matrix [A B], where A is mxm etc.
    for k in range(0,m-1):
        a = np.abs(A[k,k])
        pk = k
        for j in range(k+1,m):
            if a < np.abs(A[j,k]):
                a = np.abs(A[j,k])
                pk = j
        if pk != k:
            for p in range(k,n):
                r = A[k,p]
                A[k,p] = A[pk,p]
                A[pk,p] = r
        for j in range(k+1,m):
            l = A[j,k]/A[k,k]
            for p in range(k,n):
                A[j,p] -= l*A[k,p]
    return A


### Gaussian elimination
Let us use the Gaussian elimination algorithm on
$$A = \begin{bmatrix} 1 & 4 & 1 \\ 2 & -1 & 2 \\ 1 & 3 & 2 \end{bmatrix}.$$

In [224]:
A0 = np.array([[1.0,4,1],[2,-1,-2],[1,3,2]])
A1 = GE(A0)
print('\nWe start with \n{0}\nand use (naive) Gaussian elimination to get \n{1}'.format(np.matrix(A0),np.matrix(A1)))


We start with 
[[ 1.  4.  1.]
 [ 2. -1. -2.]
 [ 1.  3.  2.]]
and use (naive) Gaussian elimination to get 
[[ 1.          4.          1.        ]
 [ 0.         -9.         -4.        ]
 [ 0.          0.          1.44444444]]


### Gaussian elimination with partial pivoting

Next we use Gaussian elimination with partial pivoting on
$$A = \begin{bmatrix} 1 & 3 & 2 \\ 2 & -1 & 2 \\ 1 & 4 & 1 \end{bmatrix}.$$

In [225]:
B0 = np.array([[1,3,2],[2,-1,-2],[1.0,4,1]])
B1 = GEPP(A2)
print('\nWe start with \n{0}\nand use Gaussian elimination with partial pivoting to get \n{1}'.format(np.matrix(B0),np.matrix(B1)))


We start with 
[[ 1.  3.  2.]
 [ 2. -1. -2.]
 [ 1.  4.  1.]]
and use Gaussian elimination with partial pivoting to get 
[[ 2.         -1.         -2.        ]
 [ 0.          4.5         2.        ]
 [ 0.          0.          1.44444444]]


### Gaussian elimination and numerical instabilities
Now let us see how numerical instablities can be a problem for Gaussian elimination.
Let $0 < \varepsilon \ll 1$, 
$$ A = \begin{bmatrix} \varepsilon & 1 \\ 1 & 1 \end{bmatrix} \quad \text{and} \quad b = \begin{bmatrix} 1 \\ 2 \end{bmatrix}. $$
To solve the linear system $Ax = b$ we form the augmented matrix $\begin{bmatrix} A & b \end{bmatrix}$ and perform Gaussian elimination with and without pivoting to obtain repectively $\begin{bmatrix} \tilde{A}_i & \tilde{b}_i \end{bmatrix}$ for $i=1,2$, where $\tilde{A}_i$ is upper triangular.
Then we perform backward subsitution to obtain the corresponding solutions $x_1$ and $x_2$.

> Try modifying the value of $\varepsilon$ in the code and see how $x_1$ and $x_2$ change.

> Try to change the datatype to a floating point number with smaller bitsize than the standard *double* precision, e.g., *single* or *half* and see how this affects how small $\varepsilon$ needs to be before $x_1$ and $x_2$ start to differ.

In [244]:
epsi = 1e-6 #epsilon, try to vary how small it is
C0 = np.array([[epsi,1,1],[1,1,2]],dtype=np.single) # try different types double, single, half
C1 = GE(C0)
C2 = GEPP(C0)
print('We start with the augmented matrix\n{0}.\n'.format(np.matrix(C0)))
print('Naive Gaussian elimination yields\n{0},\nwhile with partial pivoting we get\n{1}.\n'.format(np.matrix(C1),np.matrix(C2)))

# backward substitution
x12 = C1[1,2]/C1[1,1]
x11 = (C1[0,2]-C1[0,1]*x12)/C1[0,0]
#x1 = np.array([x11,x12])

x22 = C2[1,2]/C2[1,1]
x21 = (C2[0,2]-C2[0,1]*x12)/C2[0,0]
#x2 = np.array([x21,x22])
display(C2.dtype)

print('Backward substitution yields\nx1=[{:.10f},{:.10f}]\nx2=[{:.10f},{:.10f}]'.format(x11,x12,x21,x22))
x = la.solve(C0[:,0:2],C0[:,2]) # built-in solver
print('\nThe built-in solver gives \nx1=[{:.10f},{:.10f}]'.format(x[0],x[1]))

We start with the augmented matrix
[[1.e-06 1.e+00 1.e+00]
 [1.e+00 1.e+00 2.e+00]].

Naive Gaussian elimination yields
[[ 1.00000e-06  1.00000e+00  1.00000e+00]
 [ 0.00000e+00 -9.99999e+05 -9.99998e+05]],
while with partial pivoting we get
[[1.       1.       2.      ]
 [0.       0.999999 0.999998]].



dtype('float32')

Backward substitution yields
x1=[1.0132789612,0.9999989867]
x2=[1.0000009537,0.9999989867]

The built-in solver gives 
x1=[1.0000009537,0.9999989867]
